In [1]:
#Connection PI
import PIconnect as PI
import xlwings as xw
import clr
from System.Net import NetworkCredential
from PIconnect.PIConsts import AuthenticationMode
from PIconnect.PIConsts import UpdateMode, BufferMode
from datetime import *

OSIsoft(r) AF SDK Version: 2.10.5.9050


C:\Users\Jteso\anaconda3\envs\Analisis\lib\site-packages\PIconnect\PIAF.py:42: InitialisationWarning: Failed loading server data for PEREGPPIAF06NT with error EndpointNotFoundException
  warnings.warn(
C:\Users\Jteso\anaconda3\envs\Analisis\lib\site-packages\PIconnect\PIAF.py:42: InitialisationWarning: Failed loading server data for EPAN03PIAF01 with error EndpointNotFoundException
  warnings.warn(
C:\Users\Jteso\anaconda3\envs\Analisis\lib\site-packages\PIconnect\PIAF.py:42: InitialisationWarning: Failed loading server data for CLSG2HX2VAF6001 with error EndpointNotFoundException
  warnings.warn(
C:\Users\Jteso\anaconda3\envs\Analisis\lib\site-packages\PIconnect\PIAF.py:42: InitialisationWarning: Failed loading server data for cocbogoc21spi03.enelint.global with error EndpointNotFoundException
  warnings.warn(
C:\Users\Jteso\anaconda3\envs\Analisis\lib\site-packages\PIconnect\PIAF.py:42: InitialisationWarning: Failed loading server data for E6AAWGBESA01 with error EndpointNotFoundExce

In [11]:
import sys
import os
# Load libraries
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Load sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pickle

import warnings
warnings.filterwarnings('ignore')

In [12]:
#LOGUEO
def logueo():
    global u,p

In [13]:
#FILTRO DE VALORES BAD
def filterarr(arr1):
    arr2=[]

    for e in arr1:
        try:
            arr2.append(float(e))
        except:    
            arr2.append(None)
    return arr2

In [24]:
#LECTURA DE ARCHIVO
def lectura_input(argument):
    
    #Lectura de tags
    df_input = pd.read_csv("./input/INV_INPUT.csv",sep=";")
    df_tags = pd.DataFrame()

    for i in df_input.columns[1:]:

        df_var = pd.DataFrame()
        df_var["TAG"] = df_input[i]

        df_var["DESCRIPTION"] = df_input["ID_WTG"].astype(str) + " - " + i

        df_tags = pd.concat([df_tags,df_var])

    df_tags = df_tags.reset_index(drop=True)
    
    date = pd.read_csv("./input/INV_INPUT_PARAMETERS.csv",sep=";")
    server = date["SERVER"].values[0]

    if(argument == "manual"):

        print("Modo manual")
        
        d_start = date["START"].values[0]
        d_start = datetime.strptime(d_start, '%d/%m/%Y')
        d_start = d_start.strftime("%Y-%m-%d %H:%M:%S")

        d_end = date["END"].values[0]
        d_end = datetime.strptime(d_end, '%d/%m/%Y')
        d_end = d_end.strftime("%Y-%m-%d %H:%M:%S")

    else: 

        d_start = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        d_end = (datetime.now() - timedelta(days = 7)).strftime("%Y-%m-%d %H:%M:%S")
    
    
    return df_tags, d_start, d_end, server

In [25]:
#DESCARGA DE DATOS DE PI

def descarga_PI(tag_fuente,d_start, d_end, server):
    
    e = tag_fuente
    
    #Proceso de descarga de datos
    if(d_start == d_end):
        print(">> Los datos horario ya están actualizados")
    else:

        PI.PIConfig.DEFAULT_TIMEZONE = 'Etc/GMT+5'
        print(datetime.today())

        df = pd.DataFrame()

        print(PI.PIConfig.DEFAULT_TIMEZONE)

        print(">> Ejecutando como modo externo ")
        #logueo()
        with PI.PIServer(server=str(server), authentication_mode=AuthenticationMode.WINDOWS_AUTHENTICATION) as server:

            print(e)
            points = server.search(e)
            data = points[0].interpolated_values(d_start, d_end, '1h')
            print(points[0])

            df["Time"]=data.index
            df[str(e)]=filterarr(data.values)

        df=df.set_index('Time')        

        df.reset_index(level=0, inplace=True)

        df['Time'] = pd.to_datetime(df['Time'])

        df=df.set_index('Time')

    return df

In [31]:
#SE OBTIENEN DATOS

def obtencion_datos_entrada(argument):
    
    df_tags, d_start, d_end, server = lectura_input(argument)
    
    df_final = pd.DataFrame()
    i=0
    error = 0

    while(i<len(df_tags)):

        print("Tag numero: " + str(i))
        df = descarga_PI(df_tags["TAG"].to_list()[i], d_start, d_end, server)
        df_r = df.copy()
        df_r.columns = [df_tags["DESCRIPTION"].to_list()[i]]
        df_final = pd.concat([df_final,df_r],axis=1)
        i+=1
        error = 0
    
    df_final.index = df_final.index.tz_localize(None)
    df_final.reset_index(inplace=True)
    
    return df_final

In [33]:
#PROCESAMIENTO

def procesamiento(names,dataset):

    df_format_predict = pd.DataFrame()

    for name in names:
        cols = []
        for i in dataset.columns:
            if (("Time" in i)|(name in i)):
                cols.append(i)
        df_format = dataset[cols]
        df_format = df_format.melt(id_vars='Time', var_name='Variable', value_name=name)
        df_format[["WTG","Variable"]] = df_format["Variable"].str.split("-",expand=True)
        df_format = df_format.drop("Variable",axis=1)

        if(len(df_format_predict)>0):
            df_format_predict = pd.merge(df_format_predict, df_format, on=['Time','WTG'], how='inner')
        else:
            df_format_predict = df_format.copy()

    return df_format_predict

In [47]:
#APLICACION DE MODELO

def aplicacion_modelo(col_input,col_ouput,df_format_predict):
    
    output_dir = "./models - CART/"
    loaded_transformers = []

    col_extract = df_format_predict.pop("WTG")
    df_format_predict.insert(1,"WTG",col_extract)
    df_format_predict = df_format_predict.dropna(subset=df_format_predict.columns[2:6])
    data_test = df_format_predict[col_input].copy()

    for i,name in enumerate(col_ouput):

        print(name)

        try:

            modelo_pkl_path = output_dir + name + ".pkl"

            with open(modelo_pkl_path, 'rb') as f:
                modelo_cargado = pickle.load(f)

            df_format_predict[name+"_model"] = modelo_cargado.predict(data_test)

            with open(os.path.join(output_dir, f'transformerYJ_{name}.pkl'), 'rb') as transformer_file:
                loaded_transformer = pickle.load(transformer_file)

            df_format_predict[name+"_model"] = loaded_transformer.inverse_transform(df_format_predict[name+"_model"].values.reshape(-1, 1)).flatten()
            
            print(f">> Listo")
        except Exception as e:
            print(f">> No se encontro el siguiente modelo o transformer YJ: {e}")
        
    return df_format_predict

In [68]:
#AJUSTE SIGMA DE INDICADOR

def indicador_salud(col_names,df_format_predict):
        
    for i in col_names[5:]:
        
        try:
            
            #Sigma o factor de ajuste
            
            if ("Transformer" in i):
                df_format_predict[i + "_sigma"] = df_format_predict[i+"_model"]*1.08 #1.07
            elif (("Generator" in i) | ("Slow shaft" in i)):
                df_format_predict[i + "_sigma"] = df_format_predict[i+"_model"]*1.07 #1.055
            elif (("Gearbox" in i) | ("Hydraulic oil" in i)):
                df_format_predict[i + "_sigma"] = df_format_predict[i+"_model"]*1.068 #MAE*2.5
            elif ("Exchanger" in i):
                df_format_predict[i + "_sigma"] = df_format_predict[i+"_model"]*1.06

            #Indicador de sobretemperatura
            df_format_predict[ i + "_HI"] = 0
            p75 = df_format_predict[i].quantile(0.75)
            df_format_predict.loc[(df_format_predict[i] > df_format_predict[i+"_sigma"]) & (df_format_predict[i]>p75), i + "_HI"] = 1

        except Exception as e:
            
            print(f">> No se encontró variable: {e}")
            continue
        
    return df_format_predict

In [69]:
# #TRF
# gearbox solo 1
# #Radiador
# 2 se;ales

In [44]:
if __name__ == "__main__":
    
    if len(sys.argv)>1:
        
        print(">> Argumento:",sys.argv[1])
        argument = sys.argv[1] 
    
    # try:
    
    dataset = obtencion_datos_entrada(argument)
                     
    col_names = [
    "Active Power",
    "Wind direction",
    "Wind Speed",
    "Ambient temperature",
    "Nacelle ambient temperature",
    "Slow shaft front bearing temperature",
    "Slow shaft rear bearing temperature",
    "Gearbox oil temperature",
    "Gearbox bearing temperature",
    "Hydraulic oil temperature",
    "Top Heat Exchanger Temperature", #radiador superior
    "Bottom Heat Exchanger Temperature", #radiador inferior
    "Generator winding temperature U", #3,15 a 5.9 MW
    "Generator winding temperature V",
    "Generator winding temperature W",
    "Generator slip ring temperature",
    "Generator DE bearing temperature",
    "Generator NDE bearing temperature",
    "Transformer temperature phase A",
    "Transformer temperature phase B",
    "Transformer temperature phase C",
    "Generator Cooling Water Temperature" #Refrigerante CNV
    ]
    
    col_input = ["Active Power","Wind Speed","Ambient temperature","Nacelle ambient temperature",]
    col_output = col_names[5:]

    df_format_predict = procesamiento(col_names,dataset)
    df_format_predict = aplicacion_modelo(col_input,col_output,df_format_predict)

    df_format_predict = indicador_salud(col_names,df_format_predict)
    df_format_predict.to_csv("./output/Resultado.csv", index=False)
    print(">> PROCESO COMPLETADO")

>> Argumento: -f
Tag numero: 0
2024-06-04 12:43:59.691821
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA43.MSA10.BJ001.WTRF1.TotW.mag.f
PIPoint(PEWWAYX.MA43.MSA10.BJ001.WTRF1.TotW.mag.f, WTG43 - Potencia activa total ( Rotor+Estator); Current Value: -11.0 kW)
Tag numero: 1
2024-06-04 12:44:06.374823
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA44.MSA10.BJ001.WTRF1.TotW.mag.f
PIPoint(PEWWAYX.MA44.MSA10.BJ001.WTRF1.TotW.mag.f, WTG44 - Potencia activa total ( Rotor+Estator); Current Value: 0.0 kW)
Tag numero: 2
2024-06-04 12:44:08.069087
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA45.MSA10.BJ001.WTRF1.TotW.mag.f
PIPoint(PEWWAYX.MA45.MSA10.BJ001.WTRF1.TotW.mag.f, WTG45 - Potencia activa total ( Rotor+Estator); Current Value: -12.0 kW)
Tag numero: 3
2024-06-04 12:44:09.825679
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA46.MSA10.BJ001.WTRF1.TotW.mag.f
PIPoint(PEWWAYX.MA46.MSA10.BJ001.WTRF1.TotW.mag.f, WTG46 - Potencia activa total ( Rotor+Estator); Current Va

PEWWAYX.WA2_W46_AN00087
PIPoint(PEWWAYX.WA2_W46_AN00087, WTG46 - Wind Direction Primary (raw); Current Value: 154.0 ?)
Tag numero: 34
2024-06-04 12:45:03.106310
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W47_AN00087
PIPoint(PEWWAYX.WA2_W47_AN00087, WTG47 - Wind Direction Primary (raw); Current Value: 16.0 ?)
Tag numero: 35
2024-06-04 12:45:05.069543
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W48_AN00087
PIPoint(PEWWAYX.WA2_W48_AN00087, WTG48 - Wind Direction Primary (raw); Current Value: -16.0 ?)
Tag numero: 36
2024-06-04 12:45:06.885874
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W49_AN00087
PIPoint(PEWWAYX.WA2_W49_AN00087, WTG49 - Wind Direction Primary (raw); Current Value: 154.0 ?)
Tag numero: 37
2024-06-04 12:45:08.556842
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W50_AN00087
PIPoint(PEWWAYX.WA2_W50_AN00087, WTG50 - Wind Direction Primary (raw); Current Value: Bad ?)
Tag numero: 38
2024-06-04 12:45:10.213619
Etc/GMT+5
>> Ejecutando co

PEWWAYX.MA54.UMD10.BS001.WMET2.HorWdSpd.mag.f
PIPoint(PEWWAYX.MA54.UMD10.BS001.WMET2.HorWdSpd.mag.f, WTG54 - Wind Speed Primary (raw); Current Value: 3.209162950515747 m/s)
Tag numero: 72
2024-06-04 12:46:09.674730
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA55.UMD10.BS001.WMET2.HorWdSpd.mag.f
PIPoint(PEWWAYX.MA55.UMD10.BS001.WMET2.HorWdSpd.mag.f, WTG55 - Wind Speed Primary (raw); Current Value: 0.7019150257110596 m/s)
Tag numero: 73
2024-06-04 12:46:11.315742
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA56.UMD10.BS001.WMET2.HorWdSpd.mag.f
PIPoint(PEWWAYX.MA56.UMD10.BS001.WMET2.HorWdSpd.mag.f, WTG56 - Wind Speed Primary (raw); Current Value: 0.5292973518371582 m/s)
Tag numero: 74
2024-06-04 12:46:12.975471
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.MA57.UMD10.BS001.WMET2.HorWdSpd.mag.f
PIPoint(PEWWAYX.MA57.UMD10.BS001.WMET2.HorWdSpd.mag.f, WTG57 - Wind Speed Primary (raw); Current Value: 4.760814666748047 m/s)
Tag numero: 75
2024-06-04 12:46:14.619886
Etc/GMT+5

IEC60870-5-104.Wayra.42.302.13.Value
PIPoint(IEC60870-5-104.Wayra.42.302.13.Value, Measured value, short floating point value; Current Value: 28.891401290893555 )
Tag numero: 105
2024-06-04 12:47:05.327322
Etc/GMT+5
>> Ejecutando como modo externo 
IEC60870-5-104.Wayra.42.302.13.Value
PIPoint(IEC60870-5-104.Wayra.42.302.13.Value, Measured value, short floating point value; Current Value: 28.891401290893555 )
Tag numero: 106
2024-06-04 12:47:07.039013
Etc/GMT+5
>> Ejecutando como modo externo 
IEC60870-5-104.Wayra.42.302.13.Value
PIPoint(IEC60870-5-104.Wayra.42.302.13.Value, Measured value, short floating point value; Current Value: 28.891401290893555 )
Tag numero: 107
2024-06-04 12:47:08.890058
Etc/GMT+5
>> Ejecutando como modo externo 
IEC60870-5-104.Wayra.42.302.13.Value
PIPoint(IEC60870-5-104.Wayra.42.302.13.Value, Measured value, short floating point value; Current Value: 28.891401290893555 )
Tag numero: 108
2024-06-04 12:47:10.612905
Etc/GMT+5
>> Ejecutando como modo externo 
IEC6

PEWWAYX.WA2_W64_AN00009
PIPoint(PEWWAYX.WA2_W64_AN00009, WTG64 - Temperatura g?ndola; Current Value: 33.349159240722656 ?C)
Tag numero: 142
2024-06-04 12:48:11.797847
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W65_AN00009
PIPoint(PEWWAYX.WA2_W65_AN00009, WTG65 - Temperatura g?ndola; Current Value: 31.749563217163086 ?C)
Tag numero: 143
2024-06-04 12:48:13.473400
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W66_AN00009
PIPoint(PEWWAYX.WA2_W66_AN00009, WTG66 - Temperatura g?ndola; Current Value: 33.30558776855469 ?C)
Tag numero: 144
2024-06-04 12:48:15.183188
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W67_AN00009
PIPoint(PEWWAYX.WA2_W67_AN00009, WTG67 - Temperatura g?ndola; Current Value: 32.32002258300781 ?C)
Tag numero: 145
2024-06-04 12:48:16.876374
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W68_AN00009
PIPoint(PEWWAYX.WA2_W68_AN00009, WTG68 - Temperatura g?ndola; Current Value: 33.516048431396484 ?C)
Tag numero: 146
2024-06-04 12:48:18.56

...

PEWWAYX.WA2_W47_AN00014
PIPoint(PEWWAYX.WA2_W47_AN00014, WTG47 - Temp rodamiento lado opuesto acoplamiento generador; Current Value: 31.152259826660156 ?C)
Tag numero: 515
2024-06-04 12:58:57.023833
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W48_AN00014
PIPoint(PEWWAYX.WA2_W48_AN00014, WTG48 - Temp rodamiento lado opuesto acoplamiento generador; Current Value: 30.05440902709961 ?C)
Tag numero: 516
2024-06-04 12:58:58.704815
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W49_AN00014
PIPoint(PEWWAYX.WA2_W49_AN00014, WTG49 - Temp rodamiento lado opuesto acoplamiento generador; Current Value: 31.54724884033203 ?C)
Tag numero: 517
2024-06-04 12:59:00.391316
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W50_AN00014
PIPoint(PEWWAYX.WA2_W50_AN00014, WTG50 - Temp rodamiento lado opuesto acoplamiento generador; Current Value: Bad ?C)
Tag numero: 518
2024-06-04 12:59:02.057871
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W51_AN00014
PIPoint(PEWWAYX.WA2_W51_A

PEWWAYX.WA2_W52_AN00101
PIPoint(PEWWAYX.WA2_W52_AN00101, WTG52 - Medium Voltage Transformer Coil(s) Temperature L1; Current Value: 44.38219451904297 ?C)
Tag numero: 550
2024-06-04 12:59:58.358720
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W53_AN00101
PIPoint(PEWWAYX.WA2_W53_AN00101, WTG53 - Medium Voltage Transformer Coil(s) Temperature L1; Current Value: 46.471710205078125 ?C)
Tag numero: 551
2024-06-04 13:00:00.006476
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W54_AN00101
PIPoint(PEWWAYX.WA2_W54_AN00101, WTG54 - Medium Voltage Transformer Coil(s) Temperature L1; Current Value: 48.095664978027344 ?C)
Tag numero: 552
2024-06-04 13:00:01.659057
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W55_AN00101
PIPoint(PEWWAYX.WA2_W55_AN00101, WTG55 - Medium Voltage Transformer Coil(s) Temperature L1; Current Value: 49.53635025024414 ?C)
Tag numero: 553
2024-06-04 13:00:03.325212
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W56_AN00101
PIPoint(PEWWAYX.WA

PEWWAYX.WA2_W57_AN00102
PIPoint(PEWWAYX.WA2_W57_AN00102, WTG57 - Medium Voltage Transformer Coil(s) Temperature L2; Current Value: -209.19992065429688 ?C)
Tag numero: 585
2024-06-04 13:00:59.578616
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W58_AN00102
PIPoint(PEWWAYX.WA2_W58_AN00102, WTG58 - Medium Voltage Transformer Coil(s) Temperature L2; Current Value: -209.19992065429688 ?C)
Tag numero: 586
2024-06-04 13:01:01.571705
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W59_AN00102
PIPoint(PEWWAYX.WA2_W59_AN00102, WTG59 - Medium Voltage Transformer Coil(s) Temperature L2; Current Value: -209.19992065429688 ?C)
Tag numero: 587
2024-06-04 13:01:03.259447
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W60_AN00102
PIPoint(PEWWAYX.WA2_W60_AN00102, WTG60 - Medium Voltage Transformer Coil(s) Temperature L2; Current Value: -209.19992065429688 ?C)
Tag numero: 588
2024-06-04 13:01:04.902596
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W61_AN00102
PIPoint(PEWW

PIPoint(PEWWAYX.WA2_W61_AN00103, WTG61 - Medium Voltage Transformer Coil(s) Temperature L3; Current Value: -209.19992065429688 ?C)
Tag numero: 619
2024-06-04 13:01:57.996873
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W62_AN00103
PIPoint(PEWWAYX.WA2_W62_AN00103, WTG62 - Medium Voltage Transformer Coil(s) Temperature L3; Current Value: -209.19992065429688 ?C)
Tag numero: 620
2024-06-04 13:01:59.662397
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W63_AN00103
PIPoint(PEWWAYX.WA2_W63_AN00103, WTG63 - Medium Voltage Transformer Coil(s) Temperature L3; Current Value: -209.19992065429688 ?C)
Tag numero: 621
2024-06-04 13:02:01.329458
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W64_AN00103
PIPoint(PEWWAYX.WA2_W64_AN00103, WTG64 - Medium Voltage Transformer Coil(s) Temperature L3; Current Value: -209.19992065429688 ?C)
Tag numero: 622
2024-06-04 13:02:03.005842
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W65_AN00103
PIPoint(PEWWAYX.WA2_W65_AN00103, WTG

PEWWAYX.WA2_W66_AN00071
PIPoint(PEWWAYX.WA2_W66_AN00071, WTG66 - Generator Cooling Water Temperature Generator Inlet; Current Value: 40.88132095336914 ?C)
Tag numero: 654
2024-06-04 13:02:58.221534
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W67_AN00071
PIPoint(PEWWAYX.WA2_W67_AN00071, WTG67 - Generator Cooling Water Temperature Generator Inlet; Current Value: 34.037532806396484 ?C)
Tag numero: 655
2024-06-04 13:02:59.954842
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W68_AN00071
PIPoint(PEWWAYX.WA2_W68_AN00071, WTG68 - Generator Cooling Water Temperature Generator Inlet; Current Value: 37.984039306640625 ?C)
Tag numero: 656
2024-06-04 13:03:01.813404
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W69_AN00071
PIPoint(PEWWAYX.WA2_W69_AN00071, WTG69 - Generator Cooling Water Temperature Generator Inlet; Current Value: 34.27043151855469 ?C)
Tag numero: 657
2024-06-04 13:03:03.640368
Etc/GMT+5
>> Ejecutando como modo externo 
PEWWAYX.WA2_W70_AN00071
PIPoint(PE

In [70]:
col_input = ["Active Power","Wind Speed","Ambient temperature","Nacelle ambient temperature",]
col_output = col_names[5:]

df_format_predict = procesamiento(col_names,dataset)
df_format_predict = aplicacion_modelo(col_input,col_output,df_format_predict)

df_format_predict = indicador_salud(col_names,df_format_predict)
df_format_predict.to_csv("./output/Resultado.csv", index=False)
print(">> PROCESO COMPLETADO")

Slow shaft front bearing temperature
>> Listo
Slow shaft rear bearing temperature
>> No se encontro el siguiente modelo o transformer YJ: [Errno 2] No such file or directory: './models - CART/Slow shaft rear bearing temperature.pkl'
Gearbox oil temperature
>> Listo
Gearbox bearing temperature
>> Listo
Hydraulic oil temperature
>> Listo
Top Heat Exchanger Temperature
>> Listo
Bottom Heat Exchanger Temperature
>> Listo
Generator winding temperature U
>> Listo
Generator winding temperature V
>> Listo
Generator winding temperature W
>> Listo
Generator slip ring temperature
>> Listo
Generator DE bearing temperature
>> Listo
Generator NDE bearing temperature
>> Listo
Transformer temperature phase A
>> Listo
Transformer temperature phase B
>> No se encontro el siguiente modelo o transformer YJ: [Errno 2] No such file or directory: './models - CART/Transformer temperature phase B.pkl'
Transformer temperature phase C
>> No se encontro el siguiente modelo o transformer YJ: [Errno 2] No such file

In [71]:
df_format_predict = indicador_salud(col_names,df_format_predict)
df_format_predict.to_csv("./output/Resultado.csv", index=False)
print(">> PROCESO COMPLETADO")

>> No se encontró variable: 'Slow shaft rear bearing temperature_model'
>> No se encontró variable: 'Transformer temperature phase B_model'
>> No se encontró variable: 'Transformer temperature phase C_model'
>> PROCESO COMPLETADO
